In [14]:
import sys,os
from tensorflow import keras
from tensorflow.keras import layers
# from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
# Options
filepath_out = '../model'
error = 'high_error_jitter' # low_error_jitter high_error_jitter gt
dataset = 'full_error'  # full_error partial_clean
delta = 20
size = 5000

In [16]:
# Load pre-built dataset
print(os.path.join('../tmp/training_dataset/',dataset,error,'X_train_'+str(delta)+'_'+str(size)+'.npy'))
X_train = np.load(os.path.join('../tmp/training_dataset/',dataset,error,'X_train_'+str(delta)+'_'+str(size)+'.npy'))
y_train = np.load(os.path.join('../tmp/training_dataset/',dataset,error,'y_train_'+str(delta)+'_'+str(size)+'.npy'))
X_val = np.load(os.path.join('../tmp/training_dataset/',dataset,error,'X_val_'+str(delta)+'_'+str(size)+'.npy'))
y_val = np.load(os.path.join('../tmp/training_dataset/',dataset,error,'y_val_'+str(delta)+'_'+str(size)+'.npy'))
print(X_train.shape,y_train.shape)

../tmp/training_dataset/full_error/high_error_jitter/X_train_20_5000.npy
(152641, 20, 36) (152641, 20, 36)


In [17]:
# Undercomplete Denoising Autoencoder with full error
name = "RNN_"+str(delta) + "_" + error
model = keras.Sequential(
    [
    # layers.Dense(delta*12*3,input_shape=(delta,12*3)),
    # layers.LSTM(256, return_sequences=False),  # compress entire sequence
    layers.LSTM(512, return_sequences=False,input_shape=(delta,12*3)),  # compress entire sequence
    layers.Dropout(0.1),
    layers.RepeatVector(delta),  # repeat compressed vector for decoding
    layers.LSTM(512, return_sequences=True),
    layers.Dropout(0.1),
    layers.Dense(12*3),
    layers.Reshape((delta, 12 * 3))
    ]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 512)               1124352   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 repeat_vector_2 (RepeatVect  (None, 20, 512)          0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 20, 512)           2099200   
                                                                 
 dropout_5 (Dropout)         (None, 20, 512)           0         
                                                                 
 dense_2 (Dense)             (None, 20, 36)            18468     
                                                      

In [18]:
opt = keras.optimizers.legacy.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss="mse", metrics=["mae"])


history = model.fit(X_train, y_train, 
                    epochs=80, 
                    batch_size=256, validation_data=(X_val,y_val))

model.save(os.path.join(filepath_out,name+".h5"))

Epoch 1/80
597/597 [==============================] - 4s 5ms/step - loss: 0.0918 - mae: 0.2343 - val_loss: 0.0805 - val_mae: 0.2224
Epoch 2/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0807 - mae: 0.2231 - val_loss: 0.0770 - val_mae: 0.2184
Epoch 3/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0779 - mae: 0.2199 - val_loss: 0.0750 - val_mae: 0.2161
Epoch 4/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0763 - mae: 0.2180 - val_loss: 0.0740 - val_mae: 0.2148
Epoch 5/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0753 - mae: 0.2166 - val_loss: 0.0732 - val_mae: 0.2137
Epoch 6/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0744 - mae: 0.2155 - val_loss: 0.0725 - val_mae: 0.2128
Epoch 7/80
597/597 [==============================] - 3s 4ms/step - loss: 0.0736 - mae: 0.2146 - val_loss: 0.0719 - val_mae: 0.2122
Epoch 8/80
597/597 [==============================] - 3s 4ms/step - loss: 0.

In [19]:
model.save(os.path.join(filepath_out,name+".h5"))